Ten notebook jest oceniany półautomatycznie. Nie twórz ani nie usuwaj komórek - struktura notebooka musi zostać zachowana. Odpowiedź wypełnij tam gdzie jest na to wskazane miejsce - odpowiedzi w innych miejscach nie będą sprawdzane (nie są widoczne dla sprawdzającego w systemie).

W szczególności zwróć uwagę, że usupełniłeś wszystkie miejsca `YOUR CODE HERE`, `WPISZ TWÓJ KOD TUTAJ`, "YOUR ANSWER HERE" lub "WPISZ TWOJĄ ODPOWIEDŹ TUTAJ".

# Moduł 4: Statystyczne tłumaczenie maszynowe

## Zadanie 1
Zadanie polega na zaimplementowaniu algorytmu Expectation-Maximization w modelu IBM Model 1 do przypasowywania słów. Będzie to fragment modelu, który tłumaczyć będzie z hiszpańskiego na angielski. 

UWAGA: Specjalny token "NULL" pomijamy w implementacji.

Dany jest mini-korpus równoległy angielsko-hiszpański
- "green house" "casa verde"
- "the house" "la casa"
- "the green house" "la casa verde"


In [1]:
# !python -m spacy download de_core_news_sm
# !python -m spacy download en_core_web_sm

import itertools
english = [["green","house"], ["the","house"], ["the", "green", "house"]]
spanish = [["casa", "verde"], ["la", "casa"], ["la", "casa", "verde"]]

W dalszych funkcjach przydatne może być wyznaczenie słownika czyli zbioru słów z korpusu dla danego języka.

In [2]:
from itertools import chain

def get_vocabulary(corpus):
    """
    Funkcja zwracająca listę unikalnych słów z korpusu podanego w formacie zmiennej english i spanish
    """
    return list(chain.from_iterable(corpus))

In [3]:
from nose.tools import assert_set_equal
assert_set_equal(set(get_vocabulary(english)), set(["the", "green", "house"]))

Zainicjalizuj rozkład prawdopodobieństwa tłumaczenia słów rozkładem jednorodnym. Ponieważ zależy nam na prostocie implementacji (a nie efektywności) możemy to prawdopodobieństwo zaimplementować jako zwykły słownik, który będzie przyjmował na wejściu krotkę dwóch słów. Słownik nazwij `translation_prob` z kluczami (słowo_es, słowo_en).

In [22]:
def initalize_translation_prob(corpus1, corpus2):
    return {(w_1, w_2): 1/len(corpus2) for w_2 in corpus1 for w_1 in corpus2}

translation_prob = initalize_translation_prob(set(get_vocabulary(english)), set(get_vocabulary(spanish)))

Wypisz zaincjalizowany słownik, żeby upewnić się że wynik jest prawidłowy.

In [23]:
translation_prob

{('la', 'house'): 0.3333333333333333,
 ('verde', 'house'): 0.3333333333333333,
 ('casa', 'house'): 0.3333333333333333,
 ('la', 'green'): 0.3333333333333333,
 ('verde', 'green'): 0.3333333333333333,
 ('casa', 'green'): 0.3333333333333333,
 ('la', 'the'): 0.3333333333333333,
 ('verde', 'the'): 0.3333333333333333,
 ('casa', 'the'): 0.3333333333333333}

Zaimplementuj pierwszy krok algorytmu EM. Wyznacz wartości oczekiwane zmiennych przypisania słowa we wszystkich zdaniach w korpusie (oznaczane na wykładzie jako `a`).

In [26]:
import pandas as pd

def calculate_expectation(corpora1, corpora2, translation_prob):
    """
    Procedura wykonująca krok "E" algorytmu EM
    Wynikiem powinny być wartości oczekiwane dla zmiennej przypisań słów w zdaniach 
    (reprezentacja dowolna, nieweryfikowana przez sprawdzarkę)
    """
    expectation = []

    for c2, c1, e_i in zip(corpora2, corpora1, range(len(corpora1))):
        expectation.append({})
        for w2 in c2:
            total = sum([translation_prob[(w2, w1)] for w1 in c1])
            
            for w1 in c1:
                tuple_temp = (w2, w1)
                if tuple_temp not in expectation[e_i]:
                    expectation[e_i][tuple_temp] = translation_prob[(w2, w1)] / total
    
    return expectation

assignment_expected_values = calculate_expectation(english, spanish, translation_prob)

Wypisz wartości oczekiwane zmiennych przypisań, aby zobaczyć jak wyglądają. Powinny one również prezentować całkowity brak wiedzy o przypisaniach (rozkłady jednorodne).

In [27]:
assignment_expected_values

[{('casa', 'green'): 0.5,
  ('casa', 'house'): 0.5,
  ('verde', 'green'): 0.5,
  ('verde', 'house'): 0.5},
 {('la', 'the'): 0.5,
  ('la', 'house'): 0.5,
  ('casa', 'the'): 0.5,
  ('casa', 'house'): 0.5},
 {('la', 'the'): 0.3333333333333333,
  ('la', 'green'): 0.3333333333333333,
  ('la', 'house'): 0.3333333333333333,
  ('casa', 'the'): 0.3333333333333333,
  ('casa', 'green'): 0.3333333333333333,
  ('casa', 'house'): 0.3333333333333333,
  ('verde', 'the'): 0.3333333333333333,
  ('verde', 'green'): 0.3333333333333333,
  ('verde', 'house'): 0.3333333333333333}]

Zaimplementuj drugi krok algorytmu EM. Wyznacz nowe `translation_prob` na podstawie oczekiwanych wartości zmiennych przypisań.

In [29]:
def calculate_maximization(corpora1, corpora2, assignment_expected_values):
    total_sum = {}
    trans_dict = {}
    for s in assignment_expected_values:
        for tuple_temp in s:
            w2, w1 = tuple_temp
            
            if w1 not in total_sum:
                total_sum[w1] = 0
            if tuple_temp not in trans_dict:
                trans_dict[tuple_temp] = 0

            total_sum[w1] += s[(w2, w1)]
            trans_dict[tuple_temp] += s[(w2, w1)]

    for t in trans_dict:
        trans_dict[t] /= total_sum[t[1]]

    return trans_dict


translation_prob = calculate_maximization(english, spanish, assignment_expected_values)

In [30]:
from nose.tools import assert_almost_equal
assert_almost_equal(translation_prob[('casa', 'house')], 4/9.)
assert_almost_equal(translation_prob[('la', 'house')], 5/18.)

Wywołaj w pętli 10 kroków algorytmu EM i zaobserwuj jak zmieniają się prawdopodobieństwa dla tłumacznienia "house".

In [31]:
for i in range(10):
    assignment_expected_values = calculate_expectation(english, spanish, translation_prob)
    translation_prob = calculate_maximization(english, spanish, assignment_expected_values)
    print([(i,j) for i,j in translation_prob.items() if i[1] == "house"])
    print("---")


[(('casa', 'house'), 0.4884829229547261), (('verde', 'house'), 0.255758538522637), (('la', 'house'), 0.255758538522637)]
---
[(('casa', 'house'), 0.5457026346909629), (('verde', 'house'), 0.2271486826545185), (('la', 'house'), 0.22714868265451843)]
---
[(('casa', 'house'), 0.6065598881415178), (('verde', 'house'), 0.19672005592924113), (('la', 'house'), 0.19672005592924105)]
---
[(('casa', 'house'), 0.6657430854592091), (('verde', 'house'), 0.1671284572703954), (('la', 'house'), 0.16712845727039538)]
---
[(('casa', 'house'), 0.7204578730894079), (('verde', 'house'), 0.139771063455296), (('la', 'house'), 0.139771063455296)]
---
[(('casa', 'house'), 0.7693767685977914), (('verde', 'house'), 0.1153116157011043), (('la', 'house'), 0.1153116157011043)]
---
[(('casa', 'house'), 0.812047157282144), (('verde', 'house'), 0.0939764213589279), (('la', 'house'), 0.0939764213589279)]
---
[(('casa', 'house'), 0.8485414255151226), (('verde', 'house'), 0.07572928724243876), (('la', 'house'), 0.0757292

Spróbujmy wykorzystać algorytm EM do stworzenia prawdopodobieństw $t(\text{angielski}|\text{polski})$ na poniższym korpusie

In [32]:
english2 = [["the","dog"], ["the","house"], ["the", "green", "house"]]
polish = [["pies"], ["dom"], ["zielony", "dom"]]

In [34]:
translation_prob = initalize_translation_prob(set(get_vocabulary(polish)), set(get_vocabulary(english2)))

for i in range(10):
    assignment_expected_values = calculate_expectation(polish, english2, translation_prob)
    translation_prob = calculate_maximization(polish, english2, assignment_expected_values)
    
print(translation_prob)


{('the', 'pies'): 0.5, ('dog', 'pies'): 0.5, ('the', 'dom'): 0.4998976014607412, ('house', 'dom'): 0.4998976014607412, ('the', 'zielony'): 0.1681262111738051, ('green', 'zielony'): 0.6637475776523899, ('green', 'dom'): 0.00020479707851748478, ('house', 'zielony'): 0.1681262111738051}


Sprawdź jak wyglądają prawdopodobieństwa tłumaczeń po 10 iteracjach. 

UWAGA: wynik algorytmu należy zapisać do zmiennej `translation_prob` -- to ona będzie testowana podczas oceniania

In [35]:
translation_prob

{('the', 'pies'): 0.5,
 ('dog', 'pies'): 0.5,
 ('the', 'dom'): 0.4998976014607412,
 ('house', 'dom'): 0.4998976014607412,
 ('the', 'zielony'): 0.1681262111738051,
 ('green', 'zielony'): 0.6637475776523899,
 ('green', 'dom'): 0.00020479707851748478,
 ('house', 'zielony'): 0.1681262111738051}

In [ ]:
from nose.tools import assert_almost_equal
# tu są ukryte testy

**Ćwiczenia**
- Sprawdź czy gdybyś dodał słówko `NULL` to algorytm nauczyłby się wiązać słówko `NULL` z `the`, które nie występuje w języku polskim?

In [37]:
english2 = [["the","dog"], ["the","house"], ["the", "green", "house"]]
polish2 = [["NULL", "pies"], ["NULL", "dom"], ["NULL", "zielony", "dom"]]

translation_prob = initalize_translation_prob(set(get_vocabulary(polish2)), set(get_vocabulary(english2)))

for i in range(10):
    assignment_expected_values = calculate_expectation(polish2, english2, translation_prob)
    translation_prob = calculate_maximization(polish2, english2, assignment_expected_values)
    
print(translation_prob)


{('the', 'NULL'): 0.9370242904733792, ('the', 'pies'): 0.09940427160822979, ('dog', 'NULL'): 0.0002794544868163832, ('dog', 'pies'): 0.9005957283917702, ('the', 'dom'): 0.15423765076184237, ('house', 'NULL'): 0.06249885711871718, ('house', 'dom'): 0.8430994831807286, ('the', 'zielony'): 0.023160485778361987, ('green', 'NULL'): 0.00019739792108716355, ('green', 'zielony'): 0.8932057647751056, ('green', 'dom'): 0.002662866057429016, ('house', 'zielony'): 0.08363374944653247}


In [38]:
translation_prob

{('the', 'NULL'): 0.9370242904733792,
 ('the', 'pies'): 0.09940427160822979,
 ('dog', 'NULL'): 0.0002794544868163832,
 ('dog', 'pies'): 0.9005957283917702,
 ('the', 'dom'): 0.15423765076184237,
 ('house', 'NULL'): 0.06249885711871718,
 ('house', 'dom'): 0.8430994831807286,
 ('the', 'zielony'): 0.023160485778361987,
 ('green', 'NULL'): 0.00019739792108716355,
 ('green', 'zielony'): 0.8932057647751056,
 ('green', 'dom'): 0.002662866057429016,
 ('house', 'zielony'): 0.08363374944653247}

- Jeśli wywołałbyś EM dla pierwszego korpusu równoległego (zmienne `english` i `spanish`) i dołączył tokeny `NULL` to EM tłumaczy NULL jako "casa" i "house" jako "casa" z takimi samymi prawdopodobieństwami. Dlaczego?

Zakładając, że `NULL` przypiszemy do pierwszych słów korpusu, otrzymamy w słowniku 3-krotnie wpis o możliwym tłumaczeniu słowa `casa` na `NULL`. Taka sama liczba występuje też wpisów o tłumaczeniu `casa` na `house`. To powoduje, że otrzymane wartości prawdopodobieństw z algorytmu EM są takie same.

## Zadanie 2
W tym zadaniu poznasz kolejne elementy biblioteki PyTorch. Konkretnie chodzi o operowanie na wymiarach tensorów, przełącznik `model.training` oraz mnożenie macierzowe paczek danych. Są to trzy oderwane od siebie tematy, ale wszystkie z nich będą potrzebne do implementacji kolejnego zadania.

**Przełącznik model.training**
Niektóre elementy sieci neuronowych takie jak dropout czy batch normalization inaczej działają w trakcie treningu modelu a inaczej w czasie predykcji. Korzystanie z tych warstw byłoby skomplikowane, gdyż wymagałoby innej obsługi w procedurze `forward`. Z tego powodu każdy z obiektów `nn.Module` posiada właściwość `training`, domyślnie ustawioną na `True`, z której korzystają warstwy takie jak dropout by dostosować swoje działanie.

In [39]:
import torch
import torch.nn as nn

class Model(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc = nn.Linear(1,1)
        
    def forward(self, x):
        return self.fc(x)
    
model = Model()
print( model.training )

True


Przełącznik ten możemy zmieniać przy pomocy funkcji `eval()`, przestawiając model w tryb ewaluacji.

In [40]:
model.eval()
print( model.training )

False


A przed treningiem modelu należy wywołać `train()`, przestawiając model w tryb nauki. 

In [41]:
model.train()
print( model.training )

True


W dotychczasowych implementacjach modeli nie wykorzystywaliśmy tego przełącznika, gdyż nie korzystaliśmy z warstw których działanie zależało od tego czy dokonujemy predykcji czy ewaluacji. Dobrą praktyką jest jednak zawsze wywoływanie `model.train()` przed treningiem modelu (i odpowiednio `model.eval()`).

**Widoki tensorów**

Przy implementacji różnych warstw sieci neuronowych, często konieczne jest inne ułożenie danych w wymiarach tensora tak aby zgadzały się one z formatem wejścia do warstwy kolejnej. Dotychczas wystarczało korzystanie z funkcji `.view()`, która ustawia wymiarowość tensora na tę podaną w jej argumencie.

In [42]:
dane = torch.arange(0,12)
print(dane)
dane2 = dane.view(3,4)
print(dane2)

tensor([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11])
tensor([[ 0,  1,  2,  3],
        [ 4,  5,  6,  7],
        [ 8,  9, 10, 11]])


Większość funkcji w Pytorch (które nie są operacjami in-place, końcówka nazwy `_`) zwraca zupełnie nowy tensor. Czy tak jest też tym razem?

In [43]:
dane2[1,0] = 44
print(dane)
print(dane2)

tensor([ 0,  1,  2,  3, 44,  5,  6,  7,  8,  9, 10, 11])
tensor([[ 0,  1,  2,  3],
        [44,  5,  6,  7],
        [ 8,  9, 10, 11]])


Zmiana danych w tensorze `dane2` spowodowała automatyczną zmianę danych w tensorze oryginalnym, gdyż funkcja `view()` zwraca jedynie widok do tego samego tensora. Widoki pozwalają na operowanie na tensorach w wygodnym formacie bez konieczności kopiowania danych. Oprócz `.view()` istnieją także inne funkcje, które tworzą widoki tensorów. Poznajmy kilka najpopularniejszych.

Funkcja `permute` przyjmuje na wejście permutację kolejnych liczb naturalnych od 0 do wymiarowości tensora i układa wymiary w ten właśnie sposób. Dla przykładu `dane2.permute(0,1)` zwróci identyczny widok, gdyż jako pierwszy wymiar ustawi wymiar 0, a jako drugi wymiar 1. Jednak już `dane2.permute(1,0)` dokona transpozycji macierzy.

In [44]:
print ( dane2.permute(0,1) )
print ( dane2.permute(1,0) )

tensor([[ 0,  1,  2,  3],
        [44,  5,  6,  7],
        [ 8,  9, 10, 11]])
tensor([[ 0, 44,  8],
        [ 1,  5,  9],
        [ 2,  6, 10],
        [ 3,  7, 11]])


Analogicznie działa to z większą liczbą wymiarów, w szczególności z trzema. Jeśli więc mamy ułożone dane w postaci `[długość sekwencji, wielkość paczki danych, liczba cech]` i chcemy je ułożyć tak by indeksowanie po poszczególych elementach paczki było na pierwszym wymiarze wystarczy stworzyć odpowiedni widok poprzez `permute`.

In [45]:
dane3 = dane.view(3,2,2)
print(dane3, dane3.shape)
dane3p = dane3.permute(1,0,2)
print(dane3p, dane3p.shape)
# Elementy są identyczne po odpowiednim spermutowaniu indeksów
dane3[2,1,0] == dane3p[1,2,0]

tensor([[[ 0,  1],
         [ 2,  3]],

        [[44,  5],
         [ 6,  7]],

        [[ 8,  9],
         [10, 11]]]) torch.Size([3, 2, 2])
tensor([[[ 0,  1],
         [44,  5],
         [ 8,  9]],

        [[ 2,  3],
         [ 6,  7],
         [10, 11]]]) torch.Size([2, 3, 2])


tensor(True)

Kolejną przydatną funkcją tworzącą widoki jest `unsqueeze` (i `squeeze`), która dodaje dodatkowy wymiar (o rozmiarze 1) w odpowiednim miejscu. Przykładowo wektor dane, chcielibyśmy przetworzyć jakąś warstwą, ale tak by potraktowała to jako 1-elementa paczka elementów o wymiarowości 12 (macierz 1x12). W tym celu możemy dodać sztuczny wymiar na pierwszej pozycji.

In [46]:
daneu = dane.unsqueeze(0)
print(dane)
print(daneu)
print(daneu.shape)

tensor([ 0,  1,  2,  3, 44,  5,  6,  7,  8,  9, 10, 11])
tensor([[ 0,  1,  2,  3, 44,  5,  6,  7,  8,  9, 10, 11]])
torch.Size([1, 12])


Analogicznie wykorzystaj funkcję `unsqueeze` aby uzyskać macierz 12x1 z wektora `dane`.

In [47]:
daneu2 = dane.unsqueeze(1)
print(dane)
print(daneu2)
print(daneu2.shape)

tensor([ 0,  1,  2,  3, 44,  5,  6,  7,  8,  9, 10, 11])
tensor([[ 0],
        [ 1],
        [ 2],
        [ 3],
        [44],
        [ 5],
        [ 6],
        [ 7],
        [ 8],
        [ 9],
        [10],
        [11]])
torch.Size([12, 1])


Z korzystaniem z widoków mogą się jednak wiązać problemy wydajnościowe modeli uczenia maszynowego. Wiele intensywnych obliczeniowo operacji matematycznych można efektywnie zaimplementować wykorzystując lokalność danych tj. zakładając że kolejne dane tensora/macierzy/wektora ułożone są obok siebie w pamięci. Wyobraź sobie np. warstwę liniową odczytującą cechy przykładu uczącego które przylegają do siebie w pamięci, a przykładu którego kolejne cechy, wskutek permutacji wymiarów, leżą w odległych miejscach pamięci. 

W PyTroch możemy sprawdzić czy tensor zawiera dane, które przylegają do siebie w pamięci wywołując funkcję `is_contiguous()` (konkretnie dane powinny być ułożone w pamięci po kolei od skrajnie prawego wymiaru tj. w przypadku macierzy wierszowo). Jest też możliwe wywołanie funkcji `contiguous()`, która wymusza przyległe ułożenie danych w tensorze poprzez skopiowanie danych i stworzenie nowego tensora. Pomimo kosztu pamięciowego i czasowego, przed wykonaniem szczególnie kosztownych operacji obliczeniowych, wykonanie tej operacji może się opłacać. Można też uważnie operować na tensorach, by unikać nie-przylegających tensorów ;) Zwróć uwagę, że nie każdy widok od razu powoduje brak przyległości danych.

In [48]:
print(daneu.is_contiguous())
print(dane3p.is_contiguous())

True
False


**Mnożenie macierzowe z paczkami danych**

Na koniec poznajmy przydatną funkcję `bmm` czyli *batch matrix multiplication*. Rozważmy sytuację w której mamy kolekcję 3 macierzy (a1, a2, a3) i potrzebujemy uzyskać wyniki mnożenia tych macierzy przez analogiczną kolekcję macierzy (b1, b2, b3). Możemy wykonać te mnożenia kolejno w pętli tj. a1@b1, a2@b2 i a3@b3, jednak istnieje także inna możliwość: spakować te macierze do tensorów i wykonać operację `bbm`, która zwróci nam dokładnie taki sam wynik (ale znacznie szybciej niż w przypadku iterowania).

In [49]:
A = torch.tensor([[[1,0],[0,1]],  [[2,0],[0,2]],  [[1,1],[1,1]] ])
B = torch.tensor([[[1,2],[3,4]],  [[-1,-2],[-3,-4]], [[1,2],[3,4]]])

In [50]:
result = torch.bmm(A,B)
print(result)
print(f"A {A.shape}, B {B.shape}, result {result.shape}")

tensor([[[ 1,  2],
         [ 3,  4]],

        [[-2, -4],
         [-6, -8]],

        [[ 4,  6],
         [ 4,  6]]])
A torch.Size([3, 2, 2]), B torch.Size([3, 2, 2]), result torch.Size([3, 2, 2])


## Zadanie 3
Celem tego zadania jest zaimplementowanie prostej architektury koder-dekoder dla problemu tłumaczenia maszynowego.

In [1]:
import torch
import torch.nn as nn
from torchtext.datasets import Multi30k

data = Multi30k(split='train', language_pair=('de', 'en'))
data_val = Multi30k(split='valid', language_pair=('de', 'en'))

Konieczna jest instalacja tokenizatorów dla angielskiego i niemieckiego (a wcześniej biblioteki `spacy` jeśli nie masz jej zainstalowanej).

In [5]:
!python -m spacy download de_core_news_sm
!python -m spacy download en_core_web_sm

✔ Download and installation successful
You can now load the package via spacy.load('de_core_news_sm')
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


Klasa `ParallelCorpus` jest typu `Dataset` co pozwala na jej użycie w obiektach klasy `DataLoader`. Do jej konstruktora należy podać zbiór danych, dwa kody języków (źródło, cel) oraz (opcjonalnie) dwa obiekty typu `Vocab` i liczbę `data_limit`. Jeżeli obiekty `Vocab` nie zostaną podane to obiekt `ParallelCorpus` utworzy je automatycznie na podstawie zbioru danych. Jeśli zaś nie podasz `data_limit` to zostanie wczytany cały zbiór danych, w przeciwnym razie wczytanie zostanie tylko `data_limit` par zdań. W dalszej implementacji będziesz potrzebował odczytać wielkość słownika co możesz zrobić poprzez wywołanie `len()` na obiektach `vocab_a` i `vocab_b`, które są właściwością klasy `ParallelCorpus`.

In [2]:
from helpers import ParallelCorpus

dataset = ParallelCorpus(data,'de', 'en', data_limit = 10000)
#W zbiorze walidacyjnym używamy tych samych słowników
dataset_val = ParallelCorpus(data_val, 'de', 'en', dataset.vocab_a, dataset.vocab_b)


c:\Python39\lib\site-packages\torchtext\data\utils.py:105: UserWarning: Spacy model "de" could not be loaded, trying "de_core_news_sm" instead
  warnings.warn(
c:\Python39\lib\site-packages\torchtext\data\utils.py:105: UserWarning: Spacy model "en" could not be loaded, trying "en_core_web_sm" instead
  warnings.warn(


Ponieważ w porównaniu do poprzednich zadań obliczenia mogą być bardziej czasochłonne to możesz skorzystać z możliwości karty graficznej zgodnej z CUDA (jeśli taką posiadasz). Aby obliczenia odbywały się na karcie graficznej wszystkie tensory uczestniczące w operacji (np. parametry modelu oraz dane) muszą zostać w niej zapisane. Możesz to zrobić wywołując na modelu lub tensorze funkcję `.to(device)` której argumentem jest urządzenie (cpu lub karta graficzna). Poniższa linijka pozwala na sprawdzenie czy karta graficzna zgodna z CUDA jest dostępna oraz przypisanie odpowiedniego urządzenia. 

*UWAGA* Jeśli nie jesteś w posiadaniu karty graficznej z CUDA to możesz mocno zmniejszyć `data_limit`, a z kolei jeśli ją masz to możesz go podwyższyć. Ilość danych wpływa na jakość uzyskanych wyników, jednak ocenie podlega poprawność implementacji, a nie jakość wyników. W przypadku dalszych problemów możesz także zmniejszyć pojawiające się dalej stałe `HIDDEN_DIM` i `WORD_EMBEDDING` czy wielkość paczki danych.

In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cpu


Koder w naszej implementacji to dwukierunkowa sieć rekurencyjna typu GRU, do której wejściem są kolejne tokeny przetworzone przez macierz zanurzeń.

In [4]:
WORD_EMBEDDING = 128

class Encoder(nn.Module):
    def __init__(self, vocab_size, hidden_size):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, WORD_EMBEDDING)
        self.rnn = nn.GRU(WORD_EMBEDDING, hidden_size // 2 , bidirectional = True)
        
    def forward(self, src, src_len):
        embedded = self.embedding(src)
        packed_embedded = nn.utils.rnn.pack_padded_sequence(embedded, src_len)
        packed_outputs, hidden = self.rnn(packed_embedded)         
        outputs, _ = nn.utils.rnn.pad_packed_sequence(packed_outputs) 
        hidden = torch.cat([hidden[0,:, :], hidden[1,:,:]], dim=1)
        return outputs, hidden.unsqueeze(0)

Analogicznie, dekoder jest jednokierunkową siecią rekurencyjną typu GRU o takiej samej wymiarowości stanu ukrytego co enkoder. Z tego powodu stan ukryty kodera może zostać przekazany dekoderowi bez żadnej transformacji.

Dekoder jest jednak trochę bardziej skomplikowany, gdyż w każdej iteracji na wejście otrzymuje kolejne słowo, które zostało przewidziane (!) w poprzedniej iteracji. Nie jest więc możliwe skolekcjonowanie całej sekwencji wejściowej i jednorazowe wywołanie `rnn(wejście)` tylko trzeba to robić element po elemencie. Dodatkowo wejściem do dekodera jest też wektor kontekstu, który powinien zostać stworzony przez mechanizm uwagi (obecnie nie jest on zaimplementowany).

Podsumowując: wejściem do dekodera jest słowo z poprzedniej iteracji (przetworzone przez macierz zanurzeń), wektor kontekstu oraz stan ukryty dekodera z ostatniej iteracji (oprócz pierwszej w której wejściem jest ostatni stan kodera).

Wyjście dekodera to z kolei przetłumaczone słowo, które jest w tej implementacji przewidywane na podstawie warstwy liniowej.

In [5]:
class Decoder(nn.Module):
    def __init__(self, vocab_size, hidden_size):
        super().__init__()
        # Słowo z poprzedniej iteracji jest reprezentowane zanurzeniem 
        self.vocab_size = vocab_size
        self.embedding = nn.Embedding(vocab_size, WORD_EMBEDDING)
        self.rnn = nn.GRU(hidden_size + WORD_EMBEDDING, hidden_size)
        self.fc = nn.Linear(hidden_size, vocab_size)
        
    
    def forward(self, input, hidden, encoder_outputs, src_len):
        # Funckję wywołujemy iteracyjnie tzn. tylko dla jednego słowa 
        # dla każdej z paczki przetwarzanych sekwencji
        
        # input = [batch size] (indeks tylko jednego słowa per zdanie)
        # hidden = [batch size, hidden_size]
        # encoder_outputs = [src len, batch size, hidden_size]
        # mask = [batch size, src len]
      
        embedded = self.embedding(input)
        
        #Wektor zer - w przyszłości wynik mechanizmu uwagi
        # context = [batch size, hidden_size]
        context = torch.zeros((input.shape[0], self.rnn.input_size - WORD_EMBEDDING)).to(embedded.device)
        
        #Wejście do sieci do wektor kontekstu połączony z przetwarzanym słowem
        rnn_input = torch.cat((embedded, context), dim = 1)
        
        # Wejście do sieci rekurencyjnej zawiera wymiar - długość przetwarzanej sekwencji
        # Konieczne jest więc "sztuczne" dodanie wymiaru na pierwszej (tj. zerowej) pozycji
        rnn_input = rnn_input.unsqueeze(0)
        #rnn_input = [1, batch size, word_embedding + hidden_size]
        
        _, hidden = self.rnn(rnn_input, hidden)
        
        prediction = self.fc(hidden.squeeze(0))
        
        return prediction, hidden

**Pytania sprawdzające zrozumienie implementacji**
- Dlaczego podczas zwracania stanu ukrytego kodera musieliśmy połączyć jego dwa pierwsze wymiary, aby zainicjalizować nimi dekoder?
- W implementacji kodera osobno zwróciliśmy `outputs`, oraz `hidden`. To ostatenie jeszcze musieliśmy połączyć w dłuższe wektory aby uzyskać jeden wektor stanu ukrytego dla każdego elementu paczki danych (gdyż sieć była dwukierunkowa). Czy nie prościej byłoby wziąć `outputs[-1]`? Podpowiedź: dla każdego $i>0$ `outputs[-1][i]` zawiera same zera.
- W implementacji dekodera do warstwy liniowej przekazaliśmy `hidden.squeeze(0)` tj. usunęliśmy pierwszy wymiar tensora (który miał długość 1). Czy jest to wymiar związany z długością przetwarzanej sekwencji przez sieć rekurencyjną, który sztucznie dodaliśmy kilka linijek wyżej? Podpowiedź: nie, sprawdź  dokumentację neuronu GRU w Pytorch
- Czy macierz zanurzeń dekodera i enkodera mogłaby być współdzielona? Tj. będzie zawierała reprezentacje tych samych słów?

Ostatecznie implementujemy model koder-dekoder, który łączy dwa powyższe obiekty. W szczególności ważna jest funkcja `forward`, która iteracyjnie wywołuje dekoder dla kolejno przetwarzanych słów. Bardziej konkretnie: dla poprzedniego słowa w języku docelowym. W tym miejscu mamy do wyboru dwie stragie: pierwsza to podawanie na wejście dekodera najbardziej prawdopodobne słowo które poprzednio przewidział lub zastosowanie *teacher forcing*. W tej implementacji model znajdujący się w trybie ewaluacji będzie przekazywał dekoderowi przewidziane przez niego słowa, a w fazie treningu będzie korzystał z *teacher forcing*.

Zwróć uwagę, że model zwraca całą serię wyjść z dekodera (zmienna `outputs`) - jest to konieczne by policzyć funkcję straty.

In [6]:
class EncoderDecoder(nn.Module):
    def __init__(self, encoder, decoder):
        super().__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.device = next(enc.parameters()).device
        
    def forward(self, src, src_len, tgt):       
        tgt_paded_len = tgt.shape[0]
        batch_size = tgt.shape[1]
        vocab_tgt_size = self.decoder.vocab_size
        
        #tensor to store decoder outputs
        outputs = torch.zeros(tgt_paded_len, batch_size, vocab_tgt_size).to(self.device)
        
        # Przetworzenie wejścia przez koder, kolejno uzyskane reprezentacje
        # kazdego słowa będą potrzebne do implementacji mechanizmu uwagi
        # Z kolei hidden posłuży do inicjalizacji stanu ukrytego dekodera
        encoder_outputs, hidden = self.encoder(src, src_len)
        
        #first input to the decoder is the <sos> tokens
        prev_word = tgt[0,:]
                
        for i in range(1, tgt_paded_len):          
            output, hidden = self.decoder(prev_word, hidden, encoder_outputs, src_len)
            outputs[i] = output
               
            # uczenie poprzez teaching forcing
            prev_word = tgt[i] if self.training else output.argmax(1)
            
        return outputs


Ostatecznie tworzymy obiekty typu `DataLoader` z odpowiednia funkcją tworzącą paczkę danych (nie musisz jej analizować).

In [7]:
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import DataLoader

def collate_fn(batch):
    src_batch, tgt_batch = [], []
    for sample in batch:
        src_batch.append(sample["text_a"])
        tgt_batch.append(sample["text_b"])
    src_batch = pad_sequence(src_batch, padding_value=0)
    tgt_batch = pad_sequence(tgt_batch, padding_value=0)
    lena_batch = torch.tensor([len(sample["text_a"]) for sample in batch], dtype=torch.int64)
    lenb_batch = torch.tensor([len(sample["text_b"]) for sample in batch], dtype=torch.int64)
    idx = torch.argsort(lena_batch, descending=True)
    return src_batch[:,idx].to(device), tgt_batch[:,idx].to(device), lena_batch[idx], lenb_batch[idx]

dataloader = DataLoader(dataset, batch_size=64, collate_fn=collate_fn)
dataloader_val = DataLoader(dataset_val, batch_size=256, collate_fn=collate_fn)

A także pętlę uczącą model, zwróć uwagę na ustawienie przełącznika `.train()`, stosowanie techniki normalizacji gradientu, ignorowanie w funkcji celu etykiet `<pad>` (uzupełnienie sekwencji do wspólnej długości) oraz na pominięcie pierwszego tokenu `<start>` w sekwencji docelowej. Pomimo tego, że w zbiorze danych każde zdanie zaczyna się od tokenu `<start>` i kończy się tokenem `<stop>`, i w związku z tym token `<start>` trafia na wejście dekodera w pierwszej iteracji, to dekoder od razu przewiduje kolejne słowo tłumaczonego zdania.

In [8]:
HIDDEN_DIM = 256
EPOCHS = 3

enc = Encoder(len(dataset.vocab_a), HIDDEN_DIM).to(device)
dec = Decoder(len(dataset.vocab_b), HIDDEN_DIM).to(device)
enc_dec = EncoderDecoder(enc, dec).to(device)

optimizer = torch.optim.Adam(enc_dec.parameters())
criterion = nn.CrossEntropyLoss(ignore_index = 0)

enc_dec.train()

for epoch in range(EPOCHS):
    epoch_loss = 0
    for i, batch in enumerate(dataloader):
        
        src, tgt, src_len, tgt_len = batch
        
        outputs = enc_dec(src, src_len, tgt)

        # Token START nie jest przewidywany przez dekoder
        tgt = tgt[1:].view(-1)
        
        # Tablekę outputs zaczęliśmy uzupełniać od indeksu 1
        outputs = outputs[1:].view(-1, dec.vocab_size)

        loss = criterion(outputs, tgt)
        loss.backward()
        
        # Ucinanie gradientu
        torch.nn.utils.clip_grad_norm_(enc_dec.parameters(), 2.)
        
        optimizer.step()
        optimizer.zero_grad()
        
        epoch_loss += loss   
    print(f'Epoch: {epoch+1:02} | Loss: {epoch_loss / len(dataloader):.3f}')


Epoch: 01 | Loss: 4.588
Epoch: 02 | Loss: 3.512
Epoch: 03 | Loss: 3.109


Wykonanie powyższych 3 epok uczenia może zająć na szybkim CPU ok. 1,5 minuty.

Sprawdź jak radzi sobie model z przetłumaczeniem przykładowego zdania ze zbioru uczącego (oczywiście, wynik może być  daleki od optymalnego z powodu krótkiego czasu treningu i pomniejszonego zbioru danych). Następnie sprawdź jak sobie model radzi z tłumaczeniem wybranego zdania ze zbioru walidacyjnego (obiekt `dataset_val`).

In [9]:
from helpers import translate
example_idx = 12

src = dataset[example_idx]["text_a"]
translate(src, enc_dec, dataset, device)

tgt =  dataset[example_idx]["text_b"]
print(f'Referencja = {dataset.vocab_b.lookup_tokens(tgt.numpy())}')

Żródło = ['<start>', 'Ein', 'schwarzer', 'Hund', 'und', 'ein', 'gefleckter', 'Hund', 'kämpfen', '.', '<stop>']
Tłumaczenie = ['<start>', 'A', 'black', 'dog', 'is', 'running', 'through', 'a', 'field', '.', '<stop>']
Referencja = ['<start>', 'A', 'black', 'dog', 'and', 'a', 'spotted', 'dog', 'are', 'fighting', '<stop>']



Zaimplemenuj funkcję, która policzy średnią wartość entropii krzyżowej na zbiorze walidacyjnym, która byłaby przydatna np. do wybrania odpowiedniej liczby epok uczących.
- pamiętaj o przełączeniu modelu w tryb ewaluacji
- pamiętaj o nieśledzeniu wartości gradientów `with torch.no_grad()`
- wykorzystaj już wcześniej zainicjalizowany `dataloader_val`, który iteruje po paczkach zbioru walidacyjnego
- na koniec dopisz wywołanie twojej funkcji do powyższej pętli uczącej algorytm

In [10]:
def evaluate_validation_set(model, dataloader, criterion):
    model.eval()  # Set the model to evaluation mode
    total_loss = 0

    with torch.no_grad():
        for batch in dataloader:
            src, tgt, src_len, tgt_len = batch

            outputs = model(src, src_len, tgt)
            tgt = tgt[1:].view(-1)
            outputs = outputs[1:].view(-1, model.decoder.vocab_size)

            loss = criterion(outputs, tgt)
            total_loss += loss.item()

    avg_loss = total_loss / len(dataloader)
    return avg_loss

loss = evaluate_validation_set(enc_dec, dataloader_val, criterion)
print(loss)

5.617352843284607


Rozbuduj dekoder o mechanizm uwagi (tj. o obliczanie wektora `context`) przedstawiony na wykładzie, gdzie do policzenia wag uwagi jest wykorzystany zwykły iloczyn skalarny. Stan ukryty dekodera `hidden` wykorzystaj jako wektor `q`, a stany ukryte kodera dla każdego elementu zdania wejściowego są w tensorze `encoder_outputs`.

In [26]:
class DecoderWithAttention(nn.Module):
    def __init__(self, vocab_size, hidden_size):
        super().__init__()
        # Słowo z poprzedniej iteracji jest reprezentowane zanurzeniem 
        self.vocab_size = vocab_size
        self.embedding = nn.Embedding(vocab_size, WORD_EMBEDDING)
        self.rnn = nn.GRU(hidden_size + WORD_EMBEDDING, hidden_size)
        self.fc = nn.Linear(hidden_size, vocab_size)
        
    
    def forward(self, input, hidden, encoder_outputs, src_len):
        # Funckję wywołujemy iteracyjnie tzn. tylko dla jednego słowa 
        # dla każdej z paczki przetwarzanych sekwencji
        
        # input = [batch size] (indeks tylko jednego słowa per zdanie)
        # hidden = [batch size, hidden_size]
        # encoder_outputs = [src len, batch size, hidden_size]
        # mask = [batch size, src len]
      
        embedded = self.embedding(input)
        
        h_att = hidden.permute(1,0,2)
        eo_att = encoder_outputs.permute(1,2,0)

        att = torch.bmm(h_att, eo_att)
        att = att.softmax(2)

        eo_ctx = encoder_outputs.permute(1,0,2)
        context = torch.bmm(att, eo_ctx)
        context = context.squeeze(1)
        
        # context = [batch size, hidden_size]
        
        #Wejście do sieci do wektor kontekstu połączony z przetwarzanym słowem
        rnn_input = torch.cat((embedded, context), dim = 1)
        
        # Wejście do sieci rekurencyjnej zawiera wymiar - długość przetwarzanej sekwencji
        # Konieczne jest więc "sztuczne" dodanie wymiaru na pierwszej (tj. zerowej) pozycji
        rnn_input = rnn_input.unsqueeze(0)
        #rnn_input = [1, batch size, word_embedding + hidden_size]
        
        _, hidden = self.rnn(rnn_input, hidden)
        
        prediction = self.fc(hidden.squeeze(0))
        
        return prediction, hidden

**Ćwiczenia**
- Porównaj wyniki uzyskane modelem z mechanizmem uwagi z wynikami bez niego. Czy udało się osiągnąć lepszą jakość tłumaczenia lub unika się pewnych rodzajów błędów?
- Jak wpływa dodanie mechanizmu uwagi na dynamikę uczenia się? Trwa ono dużej/krócej? Funkcja celu spada szybciej czy wolniej?
- Jakie czynniki są według Ciebie kluczowe jeśli chodzi o szybkość treningu systemów NMT a także wymagania pamięciowe dla nich?

Odpowedzi na powyższe nie musisz wpisywać.

In [27]:
HIDDEN_DIM = 256
EPOCHS = 3

enc = Encoder(len(dataset.vocab_a), HIDDEN_DIM).to(device)
dec = DecoderWithAttention(len(dataset.vocab_b), HIDDEN_DIM).to(device)
enc_dec = EncoderDecoder(enc, dec).to(device)

optimizer = torch.optim.Adam(enc_dec.parameters())
criterion = nn.CrossEntropyLoss(ignore_index = 0)

enc_dec.train()

for epoch in range(EPOCHS):
    epoch_loss = 0
    for i, batch in enumerate(dataloader):
        
        
        src, tgt, src_len, tgt_len = batch
        
        outputs = enc_dec(src, src_len, tgt)

        # Token START nie jest przewidywany przez dekoder
        tgt = tgt[1:].view(-1)
        
        # Tablekę outputs zaczęliśmy uzupełniać od indeksu 1
        outputs = outputs[1:].view(-1, dec.vocab_size)

        loss = criterion(outputs, tgt)
        loss.backward()
        
        # Ucinanie gradientu
        torch.nn.utils.clip_grad_norm_(enc_dec.parameters(), 2.)
        
        optimizer.step()
        optimizer.zero_grad()
        
        epoch_loss += loss   
    print(f'Epoch: {epoch+1:02} | Loss: {epoch_loss / len(dataloader):.3f}')

Epoch: 01 | Loss: 4.586
Epoch: 02 | Loss: 3.569
Epoch: 03 | Loss: 3.139
